In [1]:
import sys
import torch
from torch import nn

sys.path.append("../") # if running from this folder
from src.models import NAVAR
from src.models import TCN
from src.data import load_dataset

In [2]:
dataset = load_dataset('causeme', 'nonlinear-VAR_N-5_T-300')[0]
dataset

{'name': 'nonlinear-VAR_N-5_T-300',
 'data': tensor([[[-2.7631, -2.1176, -0.6967,  ...,  0.9088,  0.4479,  1.3872],
          [ 1.0702,  0.9536,  1.1921,  ...,  0.5790, -0.9674, -1.3237],
          [ 1.8066,  2.3471,  1.7101,  ...,  0.0251, -0.5003,  0.1296],
          [ 1.2085,  0.5059, -0.0125,  ...,  1.1270, -1.6082, -2.0201],
          [-0.4758, -0.2494, -0.4014,  ..., -0.3803, -0.0652, -0.2039]]])}

In [27]:
ground_truth = dataset['ground_truth']
#ground_truth = (torch.randn(5, 5) > 0).int() # fake ground truth data
ground_truth

KeyError: 'ground_truth'

In [5]:
navar_params = {
    'n_variables': 5,
    'hidden_dim': 8,
    'lambda1': 0.2 # regularization
}
tcn_params = {
    'kernel_size': 3,
    'n_blocks': 4,
    'n_layers': 2,
    'dropout': 0.2,
}

In [14]:
tcn_params = {
    'groups': 5, # number of causal variables
    'in_channels': 5,  # multiple of groups
    'out_channels': 25,  # multiple of groups
    'hidden_dim': 40,  # multiple of groups
    'kernel_size': 3,
    'n_blocks': 4,
    'n_layers': 2,
    'dropout': 0.2,
}
model = TCN(**tcn_params)

if torch.cuda.is_available():
    tensor_data = tensor_data.cuda()
    model = model.cuda()
    
model

TCN(
  (tcn): DefaultTCN(
    (network): Sequential(
      (0): TemporalBlock(
        (dropout): Dropout(p=0.2, inplace=False)
        (relu): ReLU()
        (convolutions): ModuleList(
          (0): ParametrizedConv1d(
            5, 40, kernel_size=(3,), stride=(1,), groups=5
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): _WeightNorm()
              )
            )
          )
          (1): ParametrizedConv1d(
            40, 40, kernel_size=(3,), stride=(1,), groups=5
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): _WeightNorm()
              )
            )
          )
        )
      )
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): TemporalBlock(
        (dropout): Dropout(p=0.2, inplace=False)
        (relu): ReLU()
        (convolutions): ModuleList(
          (0-1): 2 x ParametrizedConv1d(
            40, 40, kernel_size=(3,), st

In [6]:
model = NAVAR(**navar_params, **tcn_params)

if torch.cuda.is_available():
    tensor_data = tensor_data.cuda()
    model = model.cuda()
    
model

NAVAR(
  (contributions): TCN(
    (tcn): DefaultTCN(
      (network): Sequential(
        (0): TemporalBlock(
          (dropout): Dropout(p=0.2, inplace=False)
          (relu): ReLU()
          (convolutions): ModuleList(
            (0): ParametrizedConv1d(
              5, 40, kernel_size=(3,), stride=(1,), groups=5
              (parametrizations): ModuleDict(
                (weight): ParametrizationList(
                  (0): _WeightNorm()
                )
              )
            )
            (1): ParametrizedConv1d(
              40, 40, kernel_size=(3,), stride=(1,), groups=5
              (parametrizations): ModuleDict(
                (weight): ParametrizationList(
                  (0): _WeightNorm()
                )
              )
            )
          )
        )
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): TemporalBlock(
          (dropout): Dropout(p=0.2, inplace=False)
          (relu): ReLU()
          (convolutions): ModuleL

## Forward function
The forward function can return metrics (singular values such as loss and AUC) and artifacts (generated tensors in the process, such as causal_matrix etc)

In [15]:
# return metrics data (loss AUC etc.)
model(tensor_data)

tensor([[[ 0.2370, -0.0332,  0.2860,  ...,  0.2034,  0.2030,  0.0467],
         [-0.0835, -0.2283, -0.2499,  ..., -0.1297, -0.1929, -0.2674],
         [ 0.1389,  0.1386,  0.0464,  ...,  0.2297,  0.1298,  0.1301],
         ...,
         [-0.0440, -0.0708, -0.0274,  ..., -0.2234, -0.3793, -0.3538],
         [-0.1531, -0.2222,  0.0156,  ..., -0.1362, -0.2597, -0.2477],
         [-0.0348, -0.1209, -0.0138,  ...,  0.2015,  0.3637,  0.3025]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>)

In [16]:
# create_artifacts will return artifacts alongside metrics (contributions, causal matrix etc.)
model(tensor_data, create_artifacts=True)

TypeError: TCN.forward() got an unexpected keyword argument 'create_artifacts'

In [9]:
# temporal_matrix will apply a sliding window std to construct a temporal matrix
model(tensor_data, create_artifacts=True, temporal_matrix=True)

({'loss': tensor(1.5335, device='cuda:0', grad_fn=<AddBackward0>)},
 {'prediction': tensor([[[-0.5720, -0.0872,  0.0017,  ..., -0.0567,  0.3313,  0.2350],
           [-0.7680, -0.9373, -0.3441,  ..., -0.0939, -0.2571, -0.0378],
           [-0.1387, -0.0976, -0.1740,  ...,  0.0344, -0.1770, -1.2991],
           [-1.4404, -1.2722, -1.0456,  ..., -0.3899, -1.0287, -1.0718],
           [ 0.3156, -0.3681, -0.1327,  ..., -0.2541, -0.3116, -0.3377]]],
         device='cuda:0'),
  'contributions': tensor([[[[-8.1771e-02, -2.2894e-02, -1.2391e-01,  ..., -4.3021e-02,
              8.3342e-02, -5.7627e-02],
            [ 4.1650e-02,  2.3861e-01,  1.2839e-01,  ...,  3.4058e-02,
              1.5535e-01,  6.4186e-02],
            [-3.5083e-01, -1.9479e-01, -1.8385e-01,  ..., -1.5335e-01,
             -1.3711e-01, -1.8884e-01],
            [-4.3871e-01, -3.5985e-01, -5.2439e-02,  ..., -1.2998e-01,
             -2.6893e-02,  1.0229e-01],
            [ 2.5777e-01,  2.5187e-01,  2.3364e-01,  ...,  2.35

In [25]:
# ground_truth will compute AUC etc. and adds it to metrics
model(tensor_data, create_artifacts=True, ground_truth=ground_truth)

NameError: name 'model' is not defined